In [ ]:
import sys
import os

dir_notebook = os.path.dirname(os.path.abspath("__file__"))
dir_parent = os.path.dirname(dir_notebook)
if not dir_parent in sys.path:
    sys.path.append(dir_parent)

from PyQt5.QtWidgets import QMainWindow, QWidget, QVBoxLayout, QHBoxLayout, QGridLayout, QMessageBox, QApplication

from optic.gui.app_setup import setupMainWindow
from optic.gui.app_style import applyAppStyle
from optic.gui.base_layouts import makeLayoutComboBoxLabel, makeLayoutButtonGroup
from optic.gui.io_layouts import makeLayoutLoadFileExitHelp
from optic.gui.processing_image_layouts import makeLayoutMicrogliaXYCTStackRegistration
from optic.dialog.multi_session_file_loader import MultiSessionFileLoaderDialog

from optic.manager import WidgetManager, ConfigManager, DataManager, ControlManager, LayoutManager, initManagers

class CheckMultiSessionROICoordinatesGUI(QMainWindow):
    def __init__(self):
        APP_NAME = "CHECK_MULTI_SESSION_ROI_COORDINATES"
        QMainWindow.__init__(self)
        
        # Initialize managers
        self.widget_manager, self.config_manager, self.data_manager, self.control_manager, self.layout_manager = initManagers(
            WidgetManager(), ConfigManager(), DataManager(), ControlManager(), LayoutManager()
        )
        self.config_manager.setCurrentApp(APP_NAME)
        self.app_keys = self.config_manager.gui_defaults["APP_KEYS"]
        self.app_key_pri = self.app_keys[0]
        
        # Setup main window
        setupMainWindow(self, self.config_manager.gui_defaults)
        
        # Flag to track if setup is complete
        self.setup_complete = False
        
        # File paths lists
        self.list_path_fall = []
        self.list_path_roi_curation = []
        
        # Show file loader dialog first
        self.showFileLoaderDialog()
    
    def initUI(self):
        """
        Initialize main UI after files are loaded
        """
        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)
        self.layout_main = QVBoxLayout(self.central_widget)
        
        # Main UI layout (Grid)
        self.layout_main_ui = QGridLayout()
        self.layout_main.addLayout(self.layout_main_ui)
        
        # Setup layouts
        self.setupMainUILayouts()
        self.bindFuncAllWidget()
    
    def setupMainUILayouts(self):
        """
        Setup main UI layouts
        """
        self.layout_main_ui.addLayout(self.makeLayoutSectionLeftUpper(), 0, 0)
        self.layout_main_ui.addLayout(self.makeLayoutSectionRightUpper(), 0, 1)
        self.layout_main_ui.addLayout(self.makeLayoutSectionBottom(), 1, 0, 1, 2)
    
    def loadMultiSessionData(self):
        """
        Load data from multiple sessions
        
        Returns:
        --------
        success : bool
        """
        # TODO: Implement data loading logic
        try:
            for i, (path_fall, path_roi) in enumerate(zip(self.list_path_fall, self.list_path_roi_curation)):
                session_key = f"t_{i+1}"
                
                # Load Fall.mat
                success, e = self.data_manager.loadFallMat(
                    app_key=session_key,
                    path_fall=path_fall
                )
                if not success:
                    raise Exception(f"Failed to load Fall.mat for {session_key}: {e}")
                
                # TODO: Load ROICuration.mat
                
            return True
        except Exception as e:
            print(f"Error loading multi-session data: {e}")
            return False
        
    """
    makeLayout Functions: Component Level
    """
    
    def makeLayoutComponentROIView(self):
        """
        ROI view component with QGraphicsView
        Similar to Suite2pROICurationGUI's ROI view
        
        Returns:
        --------
        layout : QVBoxLayout
        """
        layout = QVBoxLayout()
        
        # Create QGraphicsView and QGraphicsScene
        view = self.widget_manager.makeWidgetView(key=self.app_key_pri)
        scene = self.widget_manager.makeWidgetScene(key=self.app_key_pri)
        view.setScene(scene)
        
        layout.addWidget(view)
        return layout
    
    def makeLayoutComponentSessionDisplaySettings(self):
        """
        Session-wise cell type display checkboxes
        
        Structure for each session:
          Session N
            ☑ Neuron  ☑ Glia  ☐ Neuropil  ...
        
        Returns:
        --------
        layout : QVBoxLayout
        """
        layout = QVBoxLayout()
        
        # Title label
        layout.addWidget(self.widget_manager.makeWidgetLabel(
            key="session_display_title",
            label="Session Display Settings"
        ))
        
        # Scroll area for sessions
        scroll_area = self.widget_manager.makeWidgetScrollArea(
            key="session_display_scroll"
        )
        widget_scroll = QWidget()
        layout_scroll = QVBoxLayout(widget_scroll)
        
        # Create checkboxes for each session
        num_sessions = len(self.list_path_fall)
        for i in range(num_sessions):
            session_key = f"session_{i+1}"
            
            # Session label
            layout_scroll.addWidget(self.widget_manager.makeWidgetLabel(
                key=f"{session_key}_label",
                label=f"Session {i+1}",
                bold=True
            ))
            
            # Cell type checkboxes (horizontal layout)
            layout_celltype = QHBoxLayout()
            
            # Get cell types from table columns
            # TODO: Need to define table_columns for this app or use a default set
            # For now, use a default cell type list
            default_celltypes = ["bg", "Neuron", "Glia", "Neuropil"]
            
            for celltype in default_celltypes:
                checkbox = self.widget_manager.makeWidgetCheckBox(
                    key=f"{session_key}_display_{celltype}",
                    label=celltype,
                    checked=True
                )
                layout_celltype.addWidget(checkbox)
            
            layout_scroll.addLayout(layout_celltype)
        
        layout_scroll.addStretch()
        scroll_area.setWidget(widget_scroll)
        layout.addWidget(scroll_area)
        
        return layout
    
    def makeLayoutComponentRegistration(self):
        """
        Image registration controls
        
        Components:
          - Reference session selector (ComboBox)
          - Background image type selector (ButtonGroup)
          - Register button
        
        Returns:
        --------
        layout : QVBoxLayout
        """
        layout = QVBoxLayout()
        
        # Background image type selector (use base_layout)
        layout.addWidget(self.widget_manager.makeWidgetLabel(
            key="registration_bg_label",
            label="Background Image Type:"
        ))
        
        bg_types = ["meanImg", "meanImgE", "max_proj", "Vcorr"]
        widget_bg_type = makeLayoutButtonGroup(
            q_widget=self,
            widget_manager=self.widget_manager,
            key_buttongroup="registration_bg_type",
            list_label_buttongroup=bg_types,
            axis="horizontal",
            set_exclusive=True,
            idx_check=0,  # Default: meanImg
            is_scroll=False
        )
        layout.addLayout(widget_bg_type)
        
        layout.addLayout(makeLayoutMicrogliaXYCTStackRegistration(
            self.widget_manager,
            1, # n_channels fixed
            3, # temporary       
            f"elastix_registration",
            f"elastix_ref_c",
            f"elastix_ref_plane_t",
            f"opacity_roi_pair",
            f"elastix_method",
            f"elastix_ref_c",
            f"elastix_ref_plane_t",          
            f"elastix_config", 
            f"elastix_run_t",   
            f"export_reg_tiff",
            f"show_roi_match",
            f"show_roi_pair",
            f"show_reg_stack",
            f"show_reg_im_roi",
            f"opacity_roi_pair",
        ))
        
        layout.addStretch()
        
        return layout
    
    """
    makeLayout Functions: Section Level
    """
    
    def makeLayoutSectionLeftUpper(self):
        """
        Left upper section: ROI view
        
        Returns:
        --------
        layout : QVBoxLayout
        """
        layout = QVBoxLayout()
        layout.addLayout(self.makeLayoutComponentROIView())
        return layout
    
    def makeLayoutSectionRightUpper(self):
        """
        Right upper section: Display settings and registration
        
        Returns:
        --------
        layout : QVBoxLayout
        """
        layout = QVBoxLayout()
        layout.addLayout(self.makeLayoutComponentSessionDisplaySettings())
        layout.addLayout(self.makeLayoutComponentRegistration())
        return layout
    
    def makeLayoutSectionBottom(self):
        """
        Bottom section: Control buttons (Load File, Help, Exit)
        Uses makeLayoutLoadFileExitHelp from optic.gui.io_layouts
        
        Returns:
        --------
        layout : QHBoxLayout
        """
        layout = makeLayoutLoadFileExitHelp(self.widget_manager)
        return layout
    
    """
    make SubWindow, Dialog Function
    """
    def showFileLoaderDialog(self):
        """
        Show multi-session file loader dialog
        ROICuration.mat files are REQUIRED for this application
        """
        dialog = MultiSessionFileLoaderDialog(
            self, 
            self.config_manager.gui_defaults,
            require_roi_curation=True
        )
        
        if dialog.exec_():
            # Dialog accepted - get file paths
            file_paths = dialog.getFilePaths()
            self.list_path_fall = file_paths['list_path_fall']
            self.list_path_roi_curation = file_paths['list_path_roi_curation']
            
            # Load data and initialize UI
            success = self.loadMultiSessionData()
            if success:
                self.initUI()
                self.setup_complete = True
                QMessageBox.information(self, "Success", "Files loaded successfully!")
            else:
                QMessageBox.warning(self, "Error", "Failed to load files.")
                self.close()
        else:
            # Dialog cancelled - close the application immediately
            self.close()


    """
    Function for bindFunc
    """
    # open 
    def reloadFiles(self):
        """
        Reopen the file loader dialog to reload files
        """
        self.showFileLoaderDialog()

    """
    bindFunc Functions
    """
    
    def bindFuncAllWidget(self):
        """
        Bind all widget functions
        """
        # Load File button - reopen the file loader dialog
        self.widget_manager.dict_button["load_file"].clicked.connect(
            self.reloadFiles
        )
        
        # Exit button
        self.widget_manager.dict_button["exit"].clicked.connect(self.close)
        
        # Help button
        # TODO: Implement help functionality
        # bindFuncHelp(q_button=self.widget_manager.dict_button["help"], url=AccessURL.HELP[self.config_manager.current_app])
        
        # Session display checkboxes
        num_sessions = len(self.list_path_fall)
        default_celltypes = ["Neuron", "Glia", "Neuropil"]
        
        for i in range(num_sessions):
            session_key = f"session_{i+1}"
            for celltype in default_celltypes:
                checkbox_key = f"{session_key}_display_{celltype}"
                if checkbox_key in self.widget_manager.dict_checkbox:
                    # TODO: Bind to display update function
                    pass
        
        # Registration controls
        if "registration_register" in self.widget_manager.dict_button:
            # TODO: Bind to registration function
            pass
    


if __name__ == "__main__":
    app = QApplication(sys.argv) if QApplication.instance() is None else QApplication.instance()
    applyAppStyle(app)
    gui = CheckMultiSessionROICoordinatesGUI()
    gui.show()
    sys.exit(app.exec_())

SystemExit: 0

c:\Users\sofut\anaconda3\envs\optic-caiman\lib\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
gui.data_manager.dict_Fall.keys()

dict_keys(['session_1'])

ModuleNotFoundError: No module named 'optic'